In [1]:
import numpy as np
from numpy import linalg as LA
import tensorflow as tf
import matplotlib.pyplot as plt
import h5py

/Users/akhiljalan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
w1_init=([[ 0.0901572 ,  0.2370918 ,  0.6920835 , -0.45759007, -0.22167274,
        -0.46439773, -0.45912468,  0.6203555 ,  0.0419175 ,  0.60444146,
         4.952244  ,  0.04192306,  0.53345317,  0.22216071, -0.24009007,
         6.301405  , -0.50758445, -0.21116066, -0.37131187, -0.22089699,
         0.04239784,  0.04331616, -0.18591626, -0.22142634, -0.4953288 ,
        -0.23889707,  0.67850924,  0.5476355 , -0.7077681 , -1.0123378 ,
         0.04195131,  0.22627208, -0.1888109 ,  0.21195143,  0.44928712,
         0.04276987, -0.20532611,  0.44252077, -0.04190878,  0.46343717,
        -0.22356562, -0.5474644 ,  0.04206235, -0.7823536 , -0.23852947,
         0.26123488, -0.2369954 , -0.25654712, -0.25827566,  0.5539032 ,
         0.22289808,  0.51685596,  1.0848937 , -0.6088887 , -0.04201594,
         0.21767725, -0.23810348, -0.4646694 , -0.53889185,  1.1317953 ,
         0.2089353 , -0.23368704, -5.6309223 , -0.2510263 ,  0.71514434,
         1.2417319 ,  5.88868   ,  0.4928691 ,  0.2434442 , -0.54655886,
         0.6717308 ,  0.44354093, -0.7333635 , -0.6745134 , -0.04279398,
        -0.7975697 ,  0.22850451, -0.25397167,  0.2451518 ,  1.1024855 ,
        -0.53172445,  0.04208738, -0.04233624,  0.8983515 ,  0.7710562 ,
        -0.2548618 , -0.21645324, -1.0170518 ,  0.9672949 , -0.23664552,
        -0.22946735,  0.63287175, -0.79163665, -0.52115196,  0.21819146,
        -0.22541553,  0.69617873,  0.73459744,  0.50693244, -0.24401082,
        -0.5940728 ,  1.3320855 , -1.140783  ,  0.23237722, -1.1244652 ,
        -5.6705046 ,  0.2540727 , -0.04189253, -0.20804366, -0.04187457,
        -0.21428825,  0.04335834,  0.96757776, -5.0284066 , -0.21626869,
        -0.540456  ,  0.51839244,  0.21898666,  0.9066629 ,  0.22020821,
        -0.50667083,  0.7983404 , -5.5656185 , -0.04212693,  0.25555643,
        -0.45822552,  0.24277431, -0.04205061,  0.15989499,  0.23738208,
         0.2237451 ,  0.24180941,  0.49051645, -0.45438182,  0.47147265,
        -0.04477705, -5.479455  ,  0.04174316,  0.2551995 ,  0.57939404,
        -0.6557258 , -0.04206115,  0.6763663 ,  0.23443314,  0.22873235,
        -0.04198467, -0.4861976 , -0.6498148 ,  0.44098404, -0.04172933]])
w2_init=([[-0.84504426],
       [-0.51247114],
       [-2.0340562 ],
       [-0.76634175],
       [ 0.61729795],
       [-0.58101785],
       [-0.6854419 ],
       [ 0.6577067 ],
       [-0.7736458 ],
       [-1.8916265 ],
       [-1.090016  ],
       [-0.873359  ],
       [ 0.42003942],
       [-0.47995704],
       [ 0.5497382 ],
       [-2.1801522 ],
       [-0.4831816 ],
       [ 0.5648663 ],
       [ 0.9415591 ],
       [ 0.78689337],
       [-0.91083336],
       [-0.9763873 ],
       [ 0.72957134],
       [ 0.5560705 ],
       [-0.4719117 ],
       [ 0.5045661 ],
       [ 0.66004866],
       [-1.5987552 ],
       [-0.4643787 ],
       [-1.9016262 ],
       [-0.96371204],
       [-0.611284  ],
       [ 0.65741754],
       [-0.5599199 ],
       [ 0.45351097],
       [-0.97737604],
       [ 0.7038435 ],
       [ 0.5943796 ],
       [ 0.9532466 ],
       [ 0.7460163 ],
       [ 0.5358916 ],
       [-0.44170648],
       [-0.9419488 ],
       [-0.67798716],
       [ 0.46497133],
       [-0.391163  ],
       [ 0.592325  ],
       [ 0.45341557],
       [ 0.43128943],
       [ 0.41603804],
       [-0.5674596 ],
       [ 0.38761157],
       [ 2.704492  ],
       [-0.80798954],
       [ 0.83548236],
       [-0.5111326 ],
       [ 0.6162054 ],
       [-0.7550416 ],
       [-0.4759281 ],
       [-2.5150294 ],
       [-0.50941396],
       [ 0.49656197],
       [-1.6215047 ],
       [ 0.47244617],
       [ 0.5376818 ],
       [ 3.9775271 ],
       [ 1.6411495 ],
       [ 0.45862758],
       [-0.47453666],
       [-0.45376387],
       [ 0.5765134 ],
       [ 0.56581146],
       [-1.1258857 ],
       [-1.0639522 ],
       [ 1.0760058 ],
       [-1.235642  ],
       [-0.53190786],
       [ 0.47500044],
       [-0.4640562 ],
       [ 2.372436  ],
       [-0.67921394],
       [-1.0515941 ],
       [ 1.1015248 ],
       [ 1.4750271 ],
       [-2.5024996 ],
       [ 0.43387246],
       [ 0.53801376],
       [-2.327031  ],
       [ 1.6461738 ],
       [ 0.4792684 ],
       [ 0.76675403],
       [ 0.4892529 ],
       [-1.1853842 ],
       [-0.38456675],
       [-0.80742   ],
       [ 0.45512152],
       [ 0.44872195],
       [-2.1801472 ],
       [ 0.67657053],
       [ 0.40404373],
       [-0.7937116 ],
       [ 0.77783364],
       [-2.4614215 ],
       [-0.6792038 ],
       [ 2.5339882 ],
       [-1.5957985 ],
       [-0.4930483 ],
       [ 0.9237745 ],
       [ 0.59356   ],
       [ 0.9956936 ],
       [ 0.47309944],
       [-0.9341501 ],
       [ 1.6710144 ],
       [ 1.1764897 ],
       [ 0.46367607],
       [-0.7061653 ],
       [ 0.46270266],
       [-0.8225886 ],
       [ 1.8290645 ],
       [-0.5919749 ],
       [-0.44208294],
       [-1.948723  ],
       [-1.3858926 ],
       [ 0.8691517 ],
       [-0.37294617],
       [-0.6558015 ],
       [-0.6871818 ],
       [ 1.0781469 ],
       [-0.87414324],
       [-0.47635847],
       [-0.5639866 ],
       [-0.47552544],
       [ 0.7286468 ],
       [-0.34246516],
       [ 0.6627983 ],
       [ 0.7922385 ],
       [-0.80032754],
       [-0.6089186 ],
       [-0.46824703],
       [ 0.40888965],
       [-0.56078476],
       [ 0.98349524],
       [ 0.48105317],
       [-0.5328922 ],
       [-0.70839876],
       [ 1.0339078 ],
       [-0.61342776],
       [-0.79129976],
       [ 0.48441455],
       [ 0.5570059 ]])

In [3]:
b1_init=([-1.49632066e-01,  2.16088206e-01,  3.65778732e+00, -1.21041000e+00,
       -1.35061651e-01, -1.29561055e+00, -1.22450840e+00, -2.32706118e+00,
       -2.15838999e-02,  3.23842049e+00,  9.99821246e-01,  5.85471094e-02,
        1.77022383e-01,  1.33129925e-01, -2.35600263e-01, -9.69530642e-01,
        7.31552601e-01, -9.77801457e-02, -1.28652573e+00,  2.19140470e-01,
        1.23102725e-01, -1.57810926e-01,  1.53959572e-01, -1.32225156e-01,
       -1.57481730e-01, -2.27377295e-01,  4.70594555e-01,  2.85312033e+00,
        3.12517256e-01,  5.74599028e+00, -2.24734023e-02,  1.56200081e-01,
       -7.49236792e-02,  9.45027769e-02, -9.54202712e-01, -1.19746946e-01,
        1.76245585e-01, -1.47855604e+00,  1.07089831e-02,  1.27336562e+00,
        2.21104607e-01, -1.81072652e-01,  8.26996788e-02, -5.77640235e-01,
       -2.25629151e-01,  3.77086610e-01, -2.16601476e-01, -3.45170379e-01,
       -3.56887221e-01, -5.90745807e-01, -2.19919547e-01, -1.86245930e+00,
        6.39037895e+00,  2.27631497e+00, -7.94772431e-02,  1.15748756e-01,
        3.03080708e-01, -1.28756618e+00, -1.78790972e-01, -5.63836622e+00,
        8.99272710e-02, -1.94928318e-01, -7.41466433e-02, -3.07720184e-01,
        2.70801663e-01,  7.34310913e+00,  8.29299539e-02, -7.81100869e-01,
        2.56538272e-01, -1.80862710e-01,  2.18636543e-01,  1.07038522e+00,
       -2.78851628e+00, -2.51557636e+00,  1.20577067e-01, -3.08366776e+00,
        1.66032478e-01, -3.27756613e-01,  2.67747581e-01, -6.31493044e+00,
       -1.79744363e+00, -4.68141548e-02,  7.84308538e-02, -5.00692749e+00,
        4.00230837e+00, -3.33558679e-01, -1.12384461e-01, -5.97595739e+00,
       -5.45763254e+00, -2.12760210e-01,  2.53413409e-01,  1.98413730e-01,
        4.21520996e+00,  6.86769903e-01, -2.12254256e-01, -1.46499500e-01,
        4.68130678e-01, -4.72452021e+00, -1.81595242e+00, -2.60216951e-01,
        2.21049786e+00, -1.94112194e+00,  6.55437994e+00, -2.68400759e-01,
        5.60166454e+00, -7.55500719e-02,  3.28553319e-01,  6.42770529e-03,
       -9.20422822e-02, -1.11987339e-02, -1.00595385e-01, -1.61407873e-01,
       -5.45945311e+00, -1.01744628e+00, -1.06990181e-01,  1.96982336e+00,
        1.70830369e-01, -2.16641054e-01,  5.29849386e+00,  1.28267542e-01,
        7.34108150e-01, -4.16245031e+00, -7.15808198e-02, -9.45318416e-02,
        3.37766856e-01, -1.21507788e+00, -3.34076196e-01,  4.01906781e-02,
       -1.60489708e-01,  2.17334837e-01,  1.42836973e-01,  2.45796412e-01,
        1.53452313e+00,  9.28530157e-01,  1.37115467e+00,  1.37233928e-01,
       -6.79805875e-02,  4.52714004e-02,  3.36023450e-01, -2.64137276e-02,
       -2.08564326e-01,  4.37483490e-02,  2.18686923e-01,  2.00063869e-01,
       -2.48323262e-01,  2.81832628e-02, -1.50140417e+00,  2.45667958e+00,
       -9.98386204e-01, -4.18332741e-02])
b2_init= ([-0.47102088])

In [4]:
# encoder_init_data = {}
# encoder_init_data['w1'] = np.asarray(w1_init)
# encoder_init_data['w2'] = np.asarray(w2_init)
# encoder_init_data['b1'] = np.asarray(b1_init)
# encoder_init_data['b2'] = np.asarray(b2_init)

In [5]:
# import shelve

In [6]:
# with shelve.open('intermediate_values') as db:
#     db['encoder_stepfn_tanh_id_weights'] = encoder_init_data

## Variable Learning Rates

In [7]:
batch_size = 500
num_epochs = 1000

In [8]:
def stack_weights(encoder_vars, decoder_vars): 
    #     encoder_vars, decoder_vars = entry[0], entry[1]
    for i in range(len(encoder_vars)): 
        encoder_vars[i] = np.ndarray.flatten(encoder_vars[i])
        decoder_vars[i] = np.ndarray.flatten(decoder_vars[i])
    W1, b1, W2, b2 = encoder_vars[0], encoder_vars[1], encoder_vars[2], encoder_vars[3]
    W3, b3, W4, b4 = decoder_vars[0], decoder_vars[1], decoder_vars[2], decoder_vars[3]
    return np.hstack((W1, b1, W2, b2, W3, b3, W4, b4))

In [9]:
def weight_norm_update(weight_stack_1, weight_stack_2): 
    weight_diff = weight_stack_1 - weight_stack_2
    l2_norm = LA.norm(weight_diff) / LA.norm(weight_stack_1)
    l1_norm = np.sum(np.abs(weight_diff)) / np.sum(np.abs(weight_stack_1))
    return l1_norm, l2_norm

### Define Computation Graph

In [10]:
k_squared = 0.04
x0 = tf.placeholder(tf.float32, [None, 1])
z = tf.placeholder(tf.float32, [None, 1])

# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(initial_value=w1_init, name='W1')
b1 = tf.Variable(b1_init, name='b1')
# and the weights connecting the hidden layer to the u1 output layer
W2 = tf.Variable(w2_init, name='W2')
b2 = tf.Variable(b2_init, name='b2')

#how to specify shape without weight initialization?
# declare weights connecting x1+z to a hidden layer
xavier_init = tf.glorot_uniform_initializer()

W3 = tf.Variable(xavier_init([1, 150]), name='W3') #todo dense layer init
b3 = tf.Variable(initial_value=tf.zeros([150]), name='b3')
# and the weights connecting the hidden layer to the u1 output layer
W4 = tf.Variable(xavier_init([150, 1]), expected_shape=[150, 1], name='W4')
b4 = tf.Variable(initial_value=tf.zeros([150]), name='b4')

In [11]:
#Encoder

# calculate the output of the hidden layer
hidden_out_1 = tf.add(tf.matmul(x0, W1), b1)
hidden_out_1 = tf.nn.tanh(hidden_out_1)

# # output layer
u1 = tf.identity(tf.add(tf.matmul(hidden_out_1, W2), b2))
# print(u1.get_shape())
# x1 = u1 + x0
x1 = u1 + x0

#Decoder
# add noise to x1

# x1_noise = x1
x1_noise = x1 + z

hidden_out_2 = tf.nn.sigmoid(tf.add(tf.matmul(x1_noise, W3), b3))
u2 = tf.identity(tf.add(tf.matmul(hidden_out_2, W4), b4))

x2 = x1 - u2

u1_cost = k_squared * tf.reduce_mean(tf.reduce_sum((u1)**2, axis=1))
x2_cost = tf.reduce_mean(tf.reduce_sum((x2)**2, axis=1))
wits_cost = x2_cost #+u1_cost

In [12]:
#goal: set a much lower learning rate for the encoder than the decoder, given that we've learned 
#a step function from the encoder. 
encoder_vars = [W1, b1, W2, b2]
decoder_vars = [W3, b3, W4, b4]
encoder_opt = tf.train.GradientDescentOptimizer(learning_rate = 0) #adam? 
decoder_opt = tf.train.GradientDescentOptimizer(learning_rate = 5e-5)

grads = tf.gradients(wits_cost, encoder_vars + decoder_vars)
grads1 = grads[:len(encoder_vars)]
grads2 = grads[len(encoder_vars):]

train_op1 = encoder_opt.apply_gradients(zip(grads1, encoder_vars))
train_op2 = decoder_opt.apply_gradients(zip(grads2, decoder_vars))
train_op = tf.group(train_op1, train_op2)

### Train and Test

In [13]:
x_stddev = 5
# x_train = np.random.normal(size=num_epochs * batch_size, scale=x_stddev)
x_test = np.linspace(-3 * x_stddev, 3 * x_stddev, num=300)
z_test = np.random.normal(size=(300,), scale = 1.0)
epoch_step = int(num_epochs/5)

In [14]:
# all_u1 = []
# all_x2 = []
# all_u2 = []
# all_y2 = []
# l1_weight_updates = []
# l2_weight_updates = []

# train_cost = []
# weights_dict = {}
# prev_weights = np.zeros(shape=(1051,))

mc_x_batch = np.random.normal(size=(5000, 1), scale = x_stddev)
mc_z_batch = np.random.normal(size=(5000, 1), scale = 1.0)
# mc_losses = []
num_random_starts = 100
for i in range(num_random_starts):
    print('RANDOM INITIALIZATION NUMBER: {}'.format(i + 1))
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        #training
        for epoch in range(num_epochs): 
            x_batch = np.random.normal(size=(batch_size, 1), scale = x_stddev)
            z_batch = np.random.normal(size=(batch_size, 1), scale = 1.0)

            _, cost,  = sess.run([train_op, wits_cost], feed_dict={x0: x_batch, z: z_batch})

#             train_cost.append(cost)

            mc_cost = sess.run([wits_cost], feed_dict={x0: mc_x_batch, z: mc_z_batch})
#             mc_losses.append(mc_cost[0])

#             next_weights = stack_weights(encoder_vars_tmp, decoder_vars_tmp)
#             l1_update, l2_update = weight_norm_update(prev_weights, next_weights)
#             l1_weight_updates.append(l1_update)
#             l2_weight_updates.append(l2_update)
#             prev_weights = next_weights

            if epoch % epoch_step == 0: 
                print('Step {}, Cost {}, MC Cost: {}'.format(epoch, cost, mc_cost[0]))

#         for i in range(len(x_test)): 
#             u1_temp, y2_temp, u2_temp, x2_temp = sess.run([u1, x1_noise, u2, x2], 
#                                                              feed_dict={x0: x_test[i].reshape(1,1), 
#                                                                         z: z_test[i].reshape(1, 1)})
#             all_u1.append(u1_temp[0][0])
#             all_x2.append(x2_temp[0][0])
#             all_y2.append(y2_temp[0][0])
#             all_u2.append(u2_temp[0][0])

RANDOM INITIALIZATION NUMBER: 1
Step 0, Cost 5153.5166015625, MC Cost: 4009.027099609375
Step 200, Cost 172.6238250732422, MC Cost: 171.6248779296875
Step 400, Cost 156.2258758544922, MC Cost: 160.26927185058594
Step 600, Cost 146.21661376953125, MC Cost: 156.01014709472656
Step 800, Cost 152.23690795898438, MC Cost: 153.36766052246094
RANDOM INITIALIZATION NUMBER: 2
Step 0, Cost 4698.70654296875, MC Cost: 3586.279541015625
Step 200, Cost 175.1435089111328, MC Cost: 171.48365783691406
Step 400, Cost 158.89662170410156, MC Cost: 160.78265380859375
Step 600, Cost 164.60678100585938, MC Cost: 157.07662963867188
Step 800, Cost 153.39418029785156, MC Cost: 153.43441772460938
RANDOM INITIALIZATION NUMBER: 3
Step 0, Cost 4249.509765625, MC Cost: 3579.974365234375
Step 200, Cost 175.38644409179688, MC Cost: 173.45758056640625
Step 400, Cost 155.5950164794922, MC Cost: 160.77479553222656
Step 600, Cost 162.29518127441406, MC Cost: 156.617431640625
Step 800, Cost 154.7482452392578, MC Cost: 153.

Step 400, Cost 148.43450927734375, MC Cost: 159.84207153320312
Step 600, Cost 154.2308349609375, MC Cost: 155.5930938720703
Step 800, Cost 147.75306701660156, MC Cost: 153.89263916015625
RANDOM INITIALIZATION NUMBER: 26
Step 0, Cost 4608.4140625, MC Cost: 3867.318359375
Step 200, Cost 155.9092254638672, MC Cost: 171.32044982910156
Step 400, Cost 164.69898986816406, MC Cost: 159.9412078857422
Step 600, Cost 166.06387329101562, MC Cost: 155.93734741210938
Step 800, Cost 149.59658813476562, MC Cost: 153.92474365234375
RANDOM INITIALIZATION NUMBER: 27
Step 0, Cost 4807.12109375, MC Cost: 3812.55126953125
Step 200, Cost 187.06248474121094, MC Cost: 171.68663024902344
Step 400, Cost 167.42454528808594, MC Cost: 160.28134155273438
Step 600, Cost 166.39306640625, MC Cost: 155.75975036621094
Step 800, Cost 151.44778442382812, MC Cost: 153.38021850585938
RANDOM INITIALIZATION NUMBER: 28
Step 0, Cost 4975.55859375, MC Cost: 4208.3134765625
Step 200, Cost 188.10195922851562, MC Cost: 171.927322387

KeyboardInterrupt: 

In [ ]:
plt.rcParams['figure.figsize'] = [12.0, 8.0]

In [ ]:
# fractional_losses = [1]
# for i in range(1, len(train_cost)): 
#     fractional_losses.append((train_cost[i] - train_cost[i-1])/train_cost[i-1])

In [ ]:
ignore_first = epoch_step * 2
epoch_num_range = np.arange(0, num_epochs)[ignore_first:]
plt.plot(epoch_num_range, mc_losses[ignore_first:], c='blue', label='MC Loss')
plt.plot(epoch_num_range, train_cost[ignore_first:], c='red', linestyle='--', label='Train Loss', alpha=0.3)
plt.xlabel('Epoch', fontsize=28)
plt.ylabel('Train Loss', fontsize=28)
plt.title('Training history', fontsize=28)
plt.legend();

In [ ]:
ignore_first = int(num_epochs/10)
epoch_num_range = np.arange(0, num_epochs)[ignore_first:]
plt.plot(epoch_num_range, mc_losses[ignore_first:], c='blue', label='MC Loss')
plt.plot(epoch_num_range, train_cost[ignore_first:], c='red', linestyle='--', label='Train Loss', alpha=0.3)
plt.xlabel('Epoch', fontsize=28)
plt.ylabel('Train Loss', fontsize=28)
plt.title('Training history', fontsize=28)
plt.legend();

In [ ]:
plt.scatter(epoch_num_range, mc_losses[ignore_first:], c='orange', label='Fractional Loss', s=2.0)
plt.plot(epoch_num_range, l1_weight_updates[ignore_first:], c='blue', label='Fractional l1 Weight Update')
plt.plot(epoch_num_range, l2_weight_updates[ignore_first:], c='red', alpha=0.3, linestyle='--', label='Fractional l2 Weight Update')
plt.xlabel('Epoch', fontsize=28)
plt.semilogy()
plt.legend();
plt.title('Fractional Update in Loss vs in Weights (log-scaled)', fontsize=20);
# plt.savefig('figs/fixed_encoder_test/with_noise_train_weight_history_logscale.png');

In [ ]:
plt.plot(epoch_num_range, mc_losses[ignore_first:], c='blue', label='Fractional Loss')
plt.plot(epoch_num_range, l1_weight_updates[ignore_first:], c='orange', label='Fractional l1 Weight Update')
plt.plot(epoch_num_range, l2_weight_updates[ignore_first:], c='green', label='Fractional l2 Weight Update')
plt.xlabel('Epoch', fontsize=28)
# plt.semilogy()
plt.legend();
plt.title('Fractional Update in Loss vs in Weights (no scaling)', fontsize=20)
# plt.savefig('figs/fixed_encoder_test/with_noise_train_weight_history.png');

In [ ]:
plt.scatter(x_test, all_x2, s=2.5)
plt.title('No y2 noise, fixed encoder, sigmoid-identity decoder')
plt.xlabel('x0', fontsize=28)
plt.ylabel('x2', fontsize=28)
# plt.savefig('figs/fixed_encoder_test/with_noise_x0_vs_x2.png');

In [ ]:
plt.scatter(x_test, all_u1, s=2.5)
plt.xlabel('x0', fontsize=28)
plt.ylabel('u1', fontsize=28);

In [ ]:
plt.scatter(x_test, all_u1 + x_test, s=2.5)
plt.xlabel('x0', fontsize=28)
plt.ylabel('x1', fontsize=28);

In [ ]:
plt.scatter(all_y2, all_u2, s=2.5)
plt.title('No y2 noise, fixed encoder, sigmoid-identity decoder')
plt.xlabel('y2', fontsize=28)
plt.ylabel('u2', fontsize=28)
# plt.savefig('figs/fixed_encoder_test/with_noise_x0_vs_x2.png');